# Lecture 7: Building a Webapp

## Using Flask to build a web app
Author: Jurre Hageman

This lesson, you will learn how to turn your DNA modify app into an interactive web app. You can find an example of the webapp that we will build here:
http://jurrehageman.pythonanywhere.com/

As a start, it is important that you read chapter 5 from the book Head First Python.
This book will provide you with a nice introduction in builing a webapp. It will also explain you the concepts of the request response cycle.

We will use Flask to build a webapp. Flask is a microframework written in Python and will greatly simplify coding webapps. We expect that you have read chapter 5 and we will start with the basics for designing a webapp right away:

## First start in building your webapp

If you are working at your own computer, we start with installing Flask. Flask can be installed by running (skip the !):

In [ ]:
!pip3 install Flask

From the code above you can see that Flask is already installed on my computer. Now let's start a basic Flask app:

In [ ]:
from flask import Flask #import the class Flask from the module flask

app = Flask(__name__) #instantiate a Flask object using the name of the currently active module.


@app.route('/')  # name of the decorating function with the route
def hello(): #function for the webroot. Function to be decorated
    return 'Hello world!' #returns the string hello world


if __name__ == '__main__':
    app.run() #runs the app if the __name__ variable is '__main__'

Run the above code in your favorite text editor, run the code and surf to: http://localhost:5000/
You should see the message on your screen. Congratualations, this is your first webapp running in the browser.
What did just happen? <br>
The first line imported the class Flask from the python module flask.<br>
The line started with "app" instantiates an object from the Flask class. <br>
The \__name\__ value is maintained by the Python interpreter and is set to the name of the currently active module. Thus, the Flask object receives the \__name\__ variable which is set to "\__main\__" when the module is run.<br>
The @app.route('/') might be difficult to understand but the only thing you need to understand is that this is Python's decorator syntax. This way, you can augment the behaviour of a function with code from another function without having to change a line of code within your function to be augmented! The hello function is augmented with the code from the app.route() method. This way, Flask will handle the routing for you.


If you are interested in how decorators work, have a look to the code below:

In [ ]:
def decorate_function(function):
   def function_wrapper(name):
       return "DNA is composed of {}".format(function(name))
   return function_wrapper


def get_message(seq):
   return "the nucleotides {}".format(seq)


get_message = decorate_function(get_message)

print(get_message("ATCG"))

In the code above, the 'get_message' function will be augmented by the 'decorate_function'. The function 'decorate_function' takes the 'get_message' function as an argument. This actually generates a new function assigned to the variable 'get_message' (thus the get_message function name will be overwritten). Thus, the original 'get_message' function is now augmented by the 'decorate_function' without changing the code of the get_message function. The original 'get_message' function name is now overwritten by the 'get_message' variable. In other words, 'get_message' is now the augmented function. Python3 supports the use of 'syntactic sugar' for decorators that greatly simplifies the syntax:

In [ ]:
def decorate_function(function):
   def function_wrapper(name):
       return "DNA is composed of {}".format(function(name))
   return function_wrapper


@decorate_function
def get_message(seq):
   return "the nucleotides {}".format(seq)

print(get_message("ATCG"))

The @ symbol decorates the 'get_message' function with the functionality of the 'decorate_function'. It reads as "add the functionality of decorate_function to get_message". If you want to know more about decorators, see this excellent tutorial: https://www.thecodeship.com/patterns/guide-to-python-function-decorators/ 

Back to our webapp. Now we have the basic webapp funcional, we can return some HTML instead of plain text. We will return a form for the user and map it to the route '/':

In [ ]:
from flask import Flask 

app = Flask(__name__) 

@app.route('/') 
def entry_page():
    return '''
    <!doctype html>
    <html>
    <head>
    <title>Welcome to manipulate DNA sequences on the web!</title>
    </head>
    <body>
    <h2>Welcome to manipulate DNA sequences on the web!</h2>
    <form method='POST' action='/output'>
    <table>
    <p>Use this form to submit a DNA sequence:</p>
    <tr>
    <td>DNA Sequence (5'->3'):</td>
    <td><input name='sequence' type='TEXT' width='60'></td>
    </tr>
    </table>
    <p>When you're ready, click this button:</p>
    <p><input value='Manipulate DNA' type='SUBMIT'></p>
    </form>
    </body>
    </html>
    '''

if __name__ == '__main__':
    app.run()

While this approach technically works, mixing html with python code is a bad thing to do. You will soon end up in a maintenance nightmare. Flask makes use of templates using the Jinja2 engine. You can make use of the teplating engine by importing it:

In [4]:
from flask import render_template

To make use of a templating save the html code in a seperate file and use the html extension. Save all html files in a folder named 'templates'. All html templates inhirit from a base template (much like subclasses inhirrit from a base class). Make sure to put the file 'base.html' in the 'templates' folder as well. In addition, you can save css files in a folder called 'static'. This would be our code using a template:  

In [ ]:
from flask import Flask
from flask import render_template

app = Flask(__name__) 

@app.route('/') 
def entry_page():
    return render_template('entry.html',
                          the_title='Welcome to my manipulate DNA app')

if __name__ == '__main__':
    app.run()

Without going into to much detail here, the code is much cleaner. We are no longer mixing Python code with HTML! Note the use of named arguments in the render_template functioon. The templating system uses the entry.html file which extends from the base template. The welcome message is 'injected' in the template variable and the whole page is served by the webserver for the viewer. 

Now it is time to add functionality to our app.
Let's show the user the reverse of the DNA string.
We can add a route 'output' that decorates an output_page function. The resulting augmented function will display the results. The POST method is used for the data transfer. HTTP works as a request-response protocol between a client (in our case the web browser) and server. There are two HTTP Request Methods: GET and POST and we will use the POST method here. To work with a request-response cycle, we need to import request from the flask module:

In [3]:
from flask import request

In [ ]:
from flask import Flask
from flask import render_template
from flask import request

app = Flask(__name__) 


@app.route('/output', methods=['POST'])
def output_page():
    dna_seq = request.form['sequence'].strip().upper()
    dna_rev = dna_seq[::-1]
    return render_template('results.html',
                           the_title='DNA manipulation results',
                           the_dna_up = dna_seq,
                           the_dna_rev = dna_rev)

@app.route('/') 
def entry_page():
    return render_template('entry.html',
                          the_title='Welcome to my manipulate DNA app')

if __name__ == '__main__':
    app.run()